In [1]:
import face_model
import argparse
import cv2
import sys
import numpy as np
from tqdm import tqdm
from dataset import Dataset
import torch
from torch.utils import data
import os
import time
import scipy.io as sio
from easydict import EasyDict as edict
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import itertools

args = edict()
args.image_size = '112,112'
args.model = '../../ccf_data/model/r100-softmax-my_dataset/model,1'
# args.model = '../../pretrained_model/model-y1-test2/model,0'
args.ga_model = ''
args.gpu = 0
args.det = 0
args.flip = 0
args.threshold = 1.24
model = face_model.FaceModel(args)
s = time.time()
INPUT_DATA = r'../../raw_data/train_data/mtcnn_training/'
rootdir_list = os.listdir(INPUT_DATA)
idsdir_list = []
for region_name in rootdir_list:
    if os.path.isdir(os.path.join(INPUT_DATA, region_name)):
        class_dir_list = os.listdir(os.path.join(INPUT_DATA, region_name))
        i = 0
        for class_name in class_dir_list:
            if os.path.isdir(os.path.join(INPUT_DATA, region_name , class_name)):
                if i < 100:
                    idsdir_list.append(os.path.join(region_name , class_name))
                    i += 1
                else:
                    break
pic_pair_list = []
all_pic_list = []
for ids_dir in idsdir_list:
    pic_root_path = os.path.join(INPUT_DATA, ids_dir)
    each_pic_list= os.listdir(pic_root_path)
    each_pic_list = [pic_root_path +'/'+ item for item in each_pic_list]
    all_pic_list.extend(each_pic_list)
    ls = itertools.combinations(each_pic_list, 2)
    pic_pair_list.extend(ls)

feature_dict = {}
pbar = tqdm_notebook(total=len(all_pic_list))
for pic_path in all_pic_list:
    img1 = cv2.imread(pic_path)
    aligned1 = np.transpose(img1, (2,0,1))
    f1 = model.get_feature(aligned1)
    feature_dict[pic_path] = f1
    pbar.update(1)
    
sim_sum = 0
i = 0

pbar = tqdm_notebook(total=len(pic_pair_list))
for path_pair_tuple in pic_pair_list:
    sim = np.dot(feature_dict[path_pair_tuple[0]], feature_dict[path_pair_tuple[1]].T)
    sim = 0.5 + 0.5 *sim
#     print(path_pair_tuple , sim)
    sim_sum += sim
    pbar.update(1)
#     i += 1
#     if i > 10:
#         break

t = time.time() - s
print(sim_sum)
print('%.2f%%'%(sim_sum/len(pic_pair_list)*100))
print('total time is {}, average time is {}'.format(t, t / len(pic_pair_list)))

loading ../../ccf_data/model/r100-softmax-my_dataset/model 1


RuntimeError: simple_bind error. Arguments:
data: (1, 3, 112, 112)
[05:44:35] src/storage/storage.cc:100: Check failed: e == cudaSuccess || e == cudaErrorCudartUnloading: CUDA: no CUDA-capable device is detected
Stack trace:
  [bt] (0) /root/anaconda3/envs/py36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x4b04cb) [0x7fda13a4c4cb]
  [bt] (1) /root/anaconda3/envs/py36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2e65bc6) [0x7fda16401bc6]
  [bt] (2) /root/anaconda3/envs/py36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2e68838) [0x7fda16404838]
  [bt] (3) /root/anaconda3/envs/py36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2e6af02) [0x7fda16406f02]
  [bt] (4) /root/anaconda3/envs/py36/lib/python3.6/site-packages/mxnet/libmxnet.so(mxnet::NDArray::NDArray(mxnet::TShape const&, mxnet::Context, bool, int)+0x5d0) [0x7fda15acd6d0]
  [bt] (5) /root/anaconda3/envs/py36/lib/python3.6/site-packages/mxnet/libmxnet.so(mxnet::common::InitZeros(mxnet::NDArrayStorageType, mxnet::TShape

# model process

## r100

In [ ]:
r100 base model
48516.348304918036
状态 / 得分
0.64271629000
'82.28%'

r100-arcface-my_dataset/model 5 acc 46
48461.512387055904
'82.18%'

r100-arcface-my_dataset/model 7 acc 53
48537.74381826166
'82.31%'

r100fc-arcface-my_dataset/model 1 acc 41
48623.684077415615
'82.46%'

acc 60
状态 / 得分
0.60
48694.37010390498
'82.58%'

50095.74971077591
状态 / 得分
0.61965966000
'84.96%'


50169.149034425616
'85.08%'


../../finetune_mtcnn_data/model/r100fc-softmax-asia_face/model 1
52138.99133577198
88.42%

## no pretrain

In [ ]:
best_model all pic train ,softmax , 
状态 / 得分
0.56529713000 
48944.21865822747
'83.00%'

model acc 85
46761.99369000364
'79.30%'

In [ ]:
baseline
状态 / 得分
0.64040506000
49083.414
'83.24%'

../../pretrained_model/model-r34-amf/model 0   r34 base model 
46701.57018072647
'79.20%'

../../pretrained_model/model-r50-am-lfw/model 0  r50 base model
47821.392430259846
'81.10%'

../../pretrained_model/model-y1-test2/model 0 y1 base model
46431.96678238455
'78.74%'

In [3]:
import face_model
import argparse
import cv2
import sys
import numpy as np
from tqdm import tqdm
from dataset import Dataset
import torch
from torch.utils import data
import os
import time
import scipy.io as sio
from datetime import datetime
from easydict import EasyDict as edict
IMAGE_SIZE = 112 
def resize_image(image, height = IMAGE_SIZE, width = IMAGE_SIZE):
    top, bottom, left, right = (0, 0, 0, 0)
    #獲取影象尺寸
    h, w, _ = image.shape

    #對於長寬不相等的圖片，找到最長的一邊
    longest_edge = max(h, w)    
    #計算短邊需要增加多上畫素寬度使其與長邊等長
    if h < longest_edge:
        dh = longest_edge - h
        top = dh // 2
        bottom = dh - top
    elif w < longest_edge:
        dw = longest_edge - w
        left = dw // 2
        right = dw - left
    else:
        pass 
    BLACK = [0, 0, 0]
    #給影象增加邊界，是圖片長、寬等長，cv2.BORDER_CONSTANT指定邊界顏色由value指定
    constant = cv2.copyMakeBorder(image, top , bottom, left, right, cv2.BORDER_CONSTANT, value = BLACK)
    #調整影象大小並返回
    return cv2.resize(constant, (height, width))

def get_featurs_new(model, test_list):
    pbar = tqdm(total=len(test_list))
    for idx, img_path in enumerate(test_list):
        pbar.update(1)
        if idx == 0:
            
            img = cv2.imread(img_path)
            new_image = np.transpose(img, (1, 2, 0))[:, :, ::-1]
            new_image = resize_image(img , 112 , 112)
            aligned = np.transpose(new_image, (2,0,1))
            feature = model.get_feature(aligned)
            features = [feature]
        else:
            img = cv2.imread(img_path)
            new_image = np.transpose(img, (1, 2, 0))[:, :, ::-1]
            new_image = resize_image(img , 112 , 112)
            aligned = np.transpose(new_image, (2,0,1))
            feature = model.get_feature(aligned)
            features = np.append(features, [feature], axis=0)
            
    return features



def get_featurs(model, test_list):
    pbar = tqdm(total=len(test_list))
    for idx, img_path in enumerate(test_list):
        pbar.update(1)
        if idx == 0:
            
            img = cv2.imread(img_path)
#             img = model.get_input(img)
            aligned = np.transpose(img, (2,0,1))
            feature = model.get_feature(aligned)
#             feature = feature.detach().cpu().numpy()
            features = [feature]
        else:
#             print(img_path)
            img = cv2.imread(img_path)
#             img = model.get_input(img)
            aligned = np.transpose(img, (2,0,1))
            feature = model.get_feature(aligned)
#             feature = feature.detach().cpu().numpy()
            features = np.append(features, [feature], axis=0)
            
    return features

def cosin_metric(x1, x2):
    return np.dot(x1, x2) / (np.linalg.norm(x1) * np.linalg.norm(x2))

def get_feature_dict(test_list, features):
    fe_dict = {}
    for i, each in enumerate(test_list):
        fe_dict[each] = features[i]
    return fe_dict

args = edict()
# general
args.image_size = '112,112'
args.model = ''
args.ga_model = ''
args.gpu = 0
args.det = 0
args.flip = 0
args.threshold = 1.24
args.featured_mat = ''
args.test_data_mtcnn = 0

if args.test_data_mtcnn == 0:
    jpgs_path = r'../../test_data/'
else:
    jpgs_path = r'../../test_mtcnn_data/'

sample_sub = open('../../raw_data/submission_template.csv', 'r')  # sample submission file dir

class_list = ['Indian','Caucasian','Asian','African']
sub = open('submission_diff_clss_2.csv', 'w')

diff_class_feature_dict = {'Indian':{},
                'Caucasian':{},
                'Asian':{},
                'African':{},
              }
for class_item in class_list:
    mat_path = 'r100-softmax-'+class_item+'_'+'1_20191008_no_mtcnn.mat'
    fe_dict = sio.loadmat(mat_path)
    print(f'{class_item} Output number:', len(fe_dict))
    diff_class_feature_dict[class_item] = fe_dict
    

######## cal_submission.py #########
    
lines = sample_sub.readlines()
pbar = tqdm(total=len(lines))
for line in lines:
    pair = line.split(',')[0]
    sub.write(pair + ',')
    a, b = pair.split(':')
    a_feature = diff_class_feature_dict[a.split('_')[0]][a]
    b_feature = diff_class_feature_dict[b.split('_')[0]][b]
    score = '%.5f' % (0.5 + 0.5 * (cosin_metric(a_feature[0], b_feature[0])))
    sub.write(score + '\n')
    pbar.update(1)

sample_sub.close()
sub.close()


Indian Output number: 8051
Caucasian Output number: 8258
Asian Output number: 9422
African Output number: 7817


100%|██████████| 7048024/7048024 [02:50<00:00, 52563.22it/s]